In [1]:
## conda install requests
import requests
from bs4 import BeautifulSoup
import pprint
import csv
import pandas as pd
import numpy as np
import json

In [2]:
with open('caladapt_census_tracts_dict_ID.txt', 'r') as file:
    data=json.load(file)
data


{'6001400100': '1',
 '6001400200': '2',
 '6001400300': '3',
 '6001400400': '4',
 '6001400500': '5',
 '6001400600': '6',
 '6001400700': '7',
 '6001400800': '8',
 '6001400900': '9',
 '6001401000': '10',
 '6001401100': '11',
 '6001401200': '12',
 '6001401300': '13',
 '6001401400': '14',
 '6001401500': '15',
 '6001401600': '16',
 '6001401700': '17',
 '6001401800': '18',
 '6001402200': '19',
 '6001402400': '20',
 '6001402500': '21',
 '6001402600': '22',
 '6001402700': '23',
 '6001402800': '24',
 '6001402900': '25',
 '6001403000': '26',
 '6001403100': '27',
 '6001403300': '28',
 '6001403400': '29',
 '6001403501': '30',
 '6001403502': '31',
 '6001403600': '32',
 '6001403701': '33',
 '6001403702': '34',
 '6001403800': '35',
 '6001403900': '36',
 '6001404000': '37',
 '6001404101': '38',
 '6001404102': '39',
 '6001404200': '40',
 '6001404300': '41',
 '6001404400': '42',
 '6001404501': '43',
 '6001404502': '44',
 '6001404600': '45',
 '6001404700': '46',
 '6001404800': '47',
 '6001404900': '48',
 

In [ ]:
##Get the census tracts required
ftract={}
for x in range(81):
    url=f"https://api.cal-adapt.org/api/censustracts/?page={x+1}&pagesize=100"
    response=requests.get(url)
    r=response.json()
    for dat in r['features']:
        ftract[dat['properties']['tract']]=str(dat['id'])

In [ ]:
##saves the census tracts as a file
with open('caladapt_census_tracts_dict_ID.txt', 'w') as file:
     file.write(json.dumps(ftract))

In [ ]:
##find daily model names
url="https://api.cal-adapt.org/api/series/?pagesize=501"
response=requests.get(url)
r=response.json()
fmodel={}
for dat in r['results']:
    fmodel[dat['name']]=dat['slug']
with open('caladapt_types.txt', 'w') as file:
    file.write(json.dumps(fmodel))

In [ ]:
def daily_ct(model_slug,tract_number):
    url = f'http://api.cal-adapt.org/api/series/{model_slug}/events'
    params = {'ref': f"/api/censustracts/{ftract[tract_number]}/", 'stat':'mean'}
    headers = {'ContentType': 'json'}
    response = requests.get(url, params=params, headers=headers)
    r=response.json()
    dates=pd.to_datetime(r['index'])
    data={'temp':r['data']}
    return pd.DataFrame(data,index=dates)

def daily_ct_timerange(model_slug,tract_number,begin,end):
    begin_time=pd.to_datetime(begin)
    end_time=pd.to_datetime(end)
    whole_data=daily_ct_temp(model_slug,tract_number)
    return whole_data.loc[begin_time:end_time]

def daily_ct_year(model_slug,tract_number,year):
    begin=pd.to_datetime(f"{year}0101")
    end=pd.to_datetime(f"{year}1231")
    return daily_ct_timerange(model_slug,tract_number,begin,end)

def hist_ct_year(model_slug,tract_number,year):
    dat=daily_ct_year(model_slug,tract_number,year)['temp'].tolist()
    maxdat=int(max(dat))+3
    mindat=int(min(dat))-3
    binned=np.arange(mindat,maxdat,1)
    x=np.histogram(dat,bins=binned)
    return x